# 2. Preprocessing Data

In this notebook, we explore the contents of the downloaded dataset and perform preprocessing steps to prepare the data for downstream analysis.

**Overview**

We focus on concatenating profiles from plates containing CRISPR knockdown experiments. The workflow includes:

1. **Plate Selection**: Loading only plates with CRISPR knockdown wells from the experimental metadata
2. **Feature Space Reduction**: Using the shared feature space defined in the [JUMP-single-cell repository](https://github.com/WayScience/JUMP-single-cell)
3. **Data Concatenation**: Combining all selected plates into a single DataFrame with consistent features
4. **Metadata Preservation**: Generating a JSON record containing metadata and shared feature information for reproducibility

This preprocessing ensures all profiles share the same feature space and are ready for comparative analysis across different experimental conditions.

In [2]:
import sys
import json
import pathlib

import polars as pl

sys.path.append("../../")
from utils.data_utils import split_meta_and_features
from utils.io_utils import load_profiles

## Helper functions 

Contains helper function that pertains to this notebook.

Defining the input and output directories used throughout the notebook.

> **Note:** The shared profiles utilized here are sourced from the [JUMP-single-cell](https://github.com/WayScience/JUMP-single-cell) repository. All preprocessing and profile generation steps are performed in that repository, and this notebook focuses on downstream analysis using the generated profiles.

In [3]:
# Setting data directory
data_dir = pathlib.Path("./data").resolve(strict=True)

# Setting profiles directory
profiles_dir = (data_dir / "sc-profiles").resolve(strict=True)

# Experimental metadata
exp_metadata_path = (data_dir / "CPJUMP1-experimental-metadata.csv").resolve(strict=True)

# Setting feature selection path
shared_features_config_path = (data_dir / "feature_selected_sc_qc_features.json").resolve(strict=True)

# Make a results folder
results_dir = pathlib.Path("./results").resolve()
results_dir.mkdir(exist_ok=True)

Create a list of paths that only points crispr treated plates and load the shared features config file that can be found in this [repo](https://github.com/WayScience/JUMP-single-cell)

In [4]:
# Load experimental metadata
exp_metadata = pl.read_csv(exp_metadata_path)
crispr_plate_names = exp_metadata.select("Assay_Plate_Barcode").unique().to_series().to_list()
crispr_plate_paths = [
        (profiles_dir / f"{plate}_feature_selected_sc_qc.parquet").resolve(strict=True) for plate in crispr_plate_names
    ]
# Load shared features
with open(shared_features_config_path) as f:
    loaded_shared_features = json.load(f)

shared_features = loaded_shared_features["shared-features"]

## Preprocessing CPJUMP1 CRISPR data

Using the filtered CRISPR plate file paths and shared features configuration, we load all individual profile files and concatenate them into a single comprehensive DataFrame. This step combines data from multiple experimental plates while maintaining the consistent feature space defined by the shared features list.

The concatenation process ensures:
- All profiles use the same feature set for downstream compatibility
- Metadata columns are preserved across all plates
- Data integrity is maintained during the merge operation

In [ ]:
# Loading crispr profiles with shared features and concat into a single DataFrame
loaded_profiles = load_profiles(
    profile_dir=profiles_dir,
    specific_plates=crispr_plate_paths,
    shared_features=shared_features
)

# Add index column 
loaded_profiles = loaded_profiles.with_row_index("index")

# Split meta and features
meta_cols, features_cols = split_meta_and_features(loaded_profiles)

Saving the concatenated CRISPR profiles and feature space information

In [6]:
# Saving metadata and features of the concat profile into a json file
meta_features_dict = {
    "concat-profiles": {
        "meta-features": meta_cols,
        "shared-features": features_cols
    }
}
with open(results_dir / "concat_profiles_meta_features.json", "w") as f:
    json.dump(meta_features_dict, f, indent=4)

# Save the concated profiles
loaded_profiles.write_parquet(
    results_dir / "concat_crispr_profiles.parquet")

## Preprocessing MitoCheck data